Refer:
https://www.udemy.com/course/master-langchain-pinecone-openai-build-llm-applications/

## Project: Implementing a ChatGPT App with LangChain from Scratch



In [1]:
%pip install --upgrade -qq langchain 
%pip install --upgrade -qq langchain-community
%pip install --upgrade -qq openai 


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.11 requires langsmith<0.3,>=0.1.125, but you have langsmith 0.4.5 which is incompatible.
deepeval 1.4.7 requires grpcio~=1.63.0, but you have grpcio 1.73.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepeval 1.4.7 requires grpcio~=1.63.0, but you have grpcio 1.73.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepeval 1.4.7 requires grpcio~=1.63.0, but you ha

In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_openai import ChatOpenAI

from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [3]:


from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        SystemMessage(content='You respond only in English.'),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = prompt | llm | StrOutputParser()

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit','q', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

Machine learning is a subfield of artificial intelligence that involves developing algorithms and statistical models that allow computers to learn from and make predictions or decisions based on data without being explicitly programmed. It is a way for computers to automatically learn and improve from experience without human intervention.
--------------------------------------------------
Goodbye!


## Adding Chat Memory Using ConversationBufferMemory

In [18]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chains import LLMChain

# 1. Imports
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Create memory 
memory = ConversationBufferMemory(
    return_messages=True
)

# creating our conversation chain now
def get_memory_messages(query):
    return memory.load_memory_variables(query)['chat_history']


# 3. add  MessagesPlaceholder(variable_name='messages') to the prompt
prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), # Where the memory will be stored.
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)


conversation_chain = (
    RunnablePassthrough.assign(
        chat_history=RunnableLambda(get_memory_messages)
    ) # sends current query (input by user at runtime) and history messages to next step
      |
    prompt # creates prompt using the previous two variables
      |
    llm # generates response using the prompt from previous step
)

# # 4. Add the memory to the chain using LLMChain (memory cannot be used directly in LCEL pipes)
# chain = LLMChain(
#     llm=llm,
#     prompt=prompt,
#     memory=memory,
#     verbose=False
# )

response = conversation_chain.invoke({'content': 'Hello, how are you?'})
    

KeyError: 'chat_history'

In [13]:
print(memory.load_memory_variables({})['chat_history'])

[HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", additional_kwargs={}, response_metadata={})]


In [ ]:
while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response['text'])
    print('-' * 50)

## Saving Chat Sessions

In [3]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Import FileChatMessageHistory
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

# 2. Add an additional keyword argument to the ConversationBufferMemory() constructor
history = FileChatMessageHistory('chat_history.json')
memory = ConversationBufferMemory(
    memory_key='chat_history',
    chat_memory=history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), 
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt:  Earth mass is ...


{'content': 'Earth mass is ...', 'chat_history': [], 'text': "The Earth's mass is approximately 5.97 x 10^24 kilograms (or 1.32 x 10^25 pounds). It is the total amount of matter contained within the Earth."}
--------------------------------------------------


Your prompt:  what about the Moon and Sun?


{'content': 'what about the Moon and Sun?', 'chat_history': [HumanMessage(content='Earth mass is ...'), AIMessage(content="The Earth's mass is approximately 5.97 x 10^24 kilograms (or 1.32 x 10^25 pounds). It is the total amount of matter contained within the Earth.")], 'text': "The Moon has a mass of about 7.35 x 10^22 kilograms (or 1.62 x 10^23 pounds), which is about 1.2% of the Earth's mass. The Sun, on the other hand, has a much larger mass. Its mass is approximately 1.99 x 10^30 kilograms (or 4.39 x 10^30 pounds), which is about 332,900 times the mass of the Earth. The Sun's mass accounts for about 99.86% of the total mass of the entire solar system."}
--------------------------------------------------


Your prompt:  quit


Goodbye!


In [4]:
# The messages property contains the list of messages in order.
print(history.messages)

[HumanMessage(content='Earth mass is ...'), AIMessage(content="The Earth's mass is approximately 5.97 x 10^24 kilograms (or 1.32 x 10^25 pounds). It is the total amount of matter contained within the Earth."), HumanMessage(content='what about the Moon and Sun?'), AIMessage(content="The Moon has a mass of about 7.35 x 10^22 kilograms (or 1.62 x 10^23 pounds), which is about 1.2% of the Earth's mass. The Sun, on the other hand, has a much larger mass. Its mass is approximately 1.99 x 10^30 kilograms (or 4.39 x 10^30 pounds), which is about 332,900 times the mass of the Earth. The Sun's mass accounts for about 99.86% of the total mass of the entire solar system.")]
